In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
from sklearn import feature_extraction
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
stemmer = SnowballStemmer("english")
art=pd.read_csv('C:/Users/mohinder/Data Analysis and M.Learning/articlesexcel.csv',encoding='cp1252')
link_info=art['article_body'].values.astype('U')
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in link_info:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

#print(totalvocab_stemmed)
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized},index=totalvocab_stemmed)
#vocab_frame.head(10)

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(link_info) #fit the vectorizer to links
print(tfidf_matrix)
terms = tfidf_vectorizer.get_feature_names()
# print(terms)
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
from sklearn.cluster import KMeans

num_clusters = 3
km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
#print(clusters)
#building a wordcloud out of words in clusters
from __future__ import print_function
cloud=[]
order_centroid = km.cluster_centers_.argsort()[:,::-1]
# print(order_centroid)
for i in range(num_clusters):
    print("cluster %s words:"%i,end=' ')
    for ind in order_centroid[i, :7]:
        #print(vocab_frame.ix[terms[ind]])
        print('%s'%vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end='\n')
        cloud.append(vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0])
        

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Convert all the required text into a single string here 
#and store them in word_string

#you can specify fonts, stopwords, background color and other options

wordcloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='white',
                          width=1200,
                          height=1000
                         ).generate(str(cloud))


plt.imshow(wordcloud)
plt.axis('off')
plt.show()



Wall time: 13.1 s
  (0, 24)	0.0573330408877
  (0, 28)	0.0806072378138
  (0, 68)	0.115857137404
  (0, 48)	0.243400816543
  (0, 87)	0.0979394685405
  (0, 46)	0.251617243345
  (0, 27)	0.0800875756169
  (0, 69)	0.101271354807
  (0, 55)	0.0827547299906
  (0, 52)	0.0775841564345
  (0, 0)	0.0588384859248
  (0, 11)	0.162267211029
  (0, 53)	0.229332163551
  (0, 81)	0.497857016934
  (0, 95)	0.202542709614
  (0, 15)	0.125391615283
  (0, 90)	0.0987472757454
  (0, 82)	0.202542709614
  (0, 30)	0.0833097444133
  (0, 67)	0.391757874162
  (0, 44)	0.162267211029
  (0, 62)	0.100412524543
  (0, 26)	0.136189622485
  (0, 7)	0.093402248905
  (0, 88)	0.218987549522
  :	:
  (204, 59)	0.120117465934
  (204, 20)	0.105657594223
  (204, 56)	0.112035441738
  (204, 35)	0.163317462242
  (204, 8)	0.126578875654
  (204, 76)	0.31650538881
  (204, 6)	0.116740487526
  (204, 78)	0.182118401935
  (204, 25)	0.204821153939
  (204, 18)	0.136190322647
  (204, 49)	0.206089574469
  (204, 86)	0.109108957153
  (204, 29)	0.339075827

ModuleNotFoundError: No module named 'wordcloud'